**This tutorial is about chunking text data file, converting them into embeddings, adding and querying them from ChromaDB**

*Step 1: Install Libraries*

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 119.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.2 MB/s e

In [ ]:
!pip install langchain langchain-community
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

*Step 2: Import libraries*

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

*Step 3: Access the file*

In [ ]:
file_path = "/content/AI.txt"

*Step 4: Read the file*

In [ ]:
def process_file(file_path, chunk_size = 1000, chunk_overlap = 200):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Intialise text splitter
    splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap,
        separators = ["\n\n", "\n", " ", ""])
    chunks = splitter.split_text(text)

    # Create Document objects (Document objects are structured containers containing raw chunks and its metadata(extra information about chunks))
    documents = [Document(page_content=chunk, metadata={"chunk_id": i}) for i, chunk in enumerate(chunks)]

    print(f"Created {len(documents)} document chunks")
    return documents

*Step 5: Process the file*

In [ ]:
docs = process_file(file_path, chunk_size = 500, chunk_overlap = 100)

Created 26 document chunks


*Step 6: View the chunks*

In [ ]:
for i, doc in enumerate(docs):
    print(f"--- Chunk {i+1} ---")
    print(doc.page_content)

--- Chunk 1 ---
Title: A Comprehensive Guide to Artificial Intelligence (AI). 

Content: Introduction to Artificial Intelligence
--- Chunk 2 ---
Artificial Intelligence (AI) refers to the branch of computer science that is focused on creating machines capable of performing tasks that typically require human intelligence. These tasks include reasoning, learning, problem-solving, perception, and language understanding. Over the last few decades, AI has evolved from theoretical concepts into practical technologies that are reshaping industries globally.
--- Chunk 3 ---
The ultimate goal of AI is to develop systems that can think, learn, and adapt to new situations, similarly to the way humans do.
--- Chunk 4 ---
2. History and Evolution of AI
--- Chunk 5 ---
Early Beginnings (1950s-1960s):
The roots of AI can be traced back to the 1950s, with pioneering work from Alan Turing. His famous Turing Test, proposed in 1950, aimed to evaluate a machine's ability to exhibit intelligent behavior eq

*Step 7: Initialize embedding model*

In [ ]:
def initialize_embeddings():
    """Initialize HuggingFace embeddings model"""
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",  # Fast and good quality
        model_kwargs={'device': 'cpu'}
    )
    return embeddings

*Step 8: Define function to generate embeddings*

In [ ]:
# Generate embeddings for document chunks
def generate_embeddings(docs, embeddings_model):
    """Generate embeddings for all document chunks"""

    # Extract text from documents
    texts = [doc.page_content for doc in docs]

    # Generate embeddings
    print("Generating embeddings...")
    embeddings = embeddings_model.embed_documents(texts)

    print(f"Generated embeddings for {len(embeddings)} chunks")
    print(f"Embedding dimension: {len(embeddings[0])}")

    return embeddings

*Step 9: Define function to combine embeddings with metadata*

In [ ]:
# Create embeddings with metadata
def create_embeddings_with_metadata(docs, embeddings):
    """Combine embeddings with document metadata""" # metadata is important when we chunk multiple files, and we want to retrieve using filters from the database

    embeddings_data = []
    for doc, embedding in zip(docs, embeddings):
        embeddings_data.append({
            'text': doc.page_content,
            'embedding': embedding,
            'metadata': doc.metadata
        })

    return embeddings_data

*Step 10: Function calling for generating embeddings*

In [ ]:
# Main function to process documents to embeddings
def documents_to_embeddings(docs):

    # Initialize embeddings model
    embeddings_model = initialize_embeddings()

    # Generate embeddings
    embeddings = generate_embeddings(docs, embeddings_model)

    # Combine with metadata
    embeddings_data = create_embeddings_with_metadata(docs, embeddings)

    return embeddings_data, embeddings_model

In [ ]:
# Usage with documents
print("Converting document chunks to embeddings...")
embeddings_data, embeddings_model = documents_to_embeddings(docs)

Converting document chunks to embeddings...


/tmp/ipython-input-8-4187915140.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating embeddings...
Generated embeddings for 26 chunks
Embedding dimension: 384


*Step 11: View embedding results*

In [ ]:
print(f"\nEmbedding Results:")
print(f"Total embeddings: {len(embeddings_data)}")
print(f"Embedding dimension: {len(embeddings_data[0]['embedding'])}")


Embedding Results:
Total embeddings: 26
Embedding dimension: 384


*Step 12: View Sample embeddings*

In [ ]:
sample = embeddings_data[0]
print(f"\nSample embedding:")
print(f"Text preview: {sample['text'][:200]}...")
print(f"Metadata: {sample['metadata']}")
print(f"Embedding shape: {len(sample['embedding'])}")
print(f"First 5 embedding values: {sample['embedding'][:5]}")


Sample embedding:
Text preview: Title: A Comprehensive Guide to Artificial Intelligence (AI). 

Content: Introduction to Artificial Intelligence...
Metadata: {'chunk_id': 0}
Embedding shape: 384
First 5 embedding values: [0.024591200053691864, -0.009280803613364697, 0.02138330042362213, 0.010370465926826, -0.044209688901901245]


*Step 13: Adding the embeddings into ChromaDB*

In [ ]:
import chromadb

In [ ]:
# Create ChromaDB client
client = chromadb.Client() # This code represents begining of database engine

# Get or create a collection
collection = client.get_or_create_collection(name="my_documents") # This code represents creating space in the database to store the data

In [ ]:
# Prepare data for ChromaDB
texts = [item['text'] for item in embeddings_data]
embeddings = [item['embedding'] for item in embeddings_data]
metadatas = [item['metadata'] for item in embeddings_data]
ids = [f"chunk_{i}" for i in range(len(embeddings_data))]  # Unique IDs

In [ ]:
# Add to ChromaDB
collection.add(   # Data store in chromadb in collections, where collections are kind of structures which store embeddings, its chunks, its metdata and unique ids
    documents=texts,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)

In [ ]:
# Query Chroma with a test embedding
query_text = "What is AI?"
query_embedding = embeddings_model.embed_query(query_text)

In [ ]:
# Retrieve relevant vectors for the given query
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3) # Top 3 closest vectors for the given query vector

In [ ]:
# Print the results
print("Query Results:")
for doc in results["documents"][0]: # This code represents retrieving relevant texts for the given query, here results['documents'] will show all the relevant text for all the queries. Here we gave one query, so results['documents'][0] will give text for the first query
    print("-", doc) # results['documents'] is a list of list

Query Results:
- Artificial Intelligence (AI) refers to the branch of computer science that is focused on creating machines capable of performing tasks that typically require human intelligence. These tasks include reasoning, learning, problem-solving, perception, and language understanding. Over the last few decades, AI has evolved from theoretical concepts into practical technologies that are reshaping industries globally.
- The ultimate goal of AI is to develop systems that can think, learn, and adapt to new situations, similarly to the way humans do.
- Manufacturing and Robotics:
AI is used in robotics for tasks such as assembly, quality control, and predictive maintenance. Industrial robots can perform repetitive tasks with precision, improving efficiency and reducing human error.

Retail and E-Commerce:
AI is used in recommendation systems to suggest products to customers based on their past behavior and preferences. Chatbots assist customers in online shopping, while AI is also 